In [1]:
import spacy
from spacy import displacy

import bioprocessor
import chemicalprocessor
import diseaseprocessor
import class_entities

import re

In [2]:
colors = {"DISEASE":"linear-gradient(90deg, #aa9cfc, #fc9ce7)",
          "CHEMICAL":"linear-gradient(90deg, #43C6AC, ##F8FFAE)"}

In [18]:
disease_service = diseaseprocessor.DiseaseProcessor('./models/Disease')
print('Disease Model Loaded')

chemical_service = chemicalprocessor.ChemicalProcessor('./models/Chemical')
print('Chemical Model Loaded')

Disease Model Loaded
Chemical Model Loaded


In [12]:
with open('prueba_comb.txt','r') as f:
    sequence = f.read()
    print(sequence)

The development of a portfolio of COVID-19 vaccines to vaccinate the global population remains an urgent public health imperative1. Here we demonstrate the capacity of a subunit vaccine, comprising the SARS-CoV-2 spike receptor binding domain displayed on a protein nanoparticle (RBD-NP), to stimulate robust and durable neutralizing antibody (nAb) responses and protection against SARS-CoV-2 in non-human primates. We evaluated five adjuvants including Essai O/W 1849101, a squalene-in-water emulsion; AS03, an alpha-tocopherol-containing oil-in-water emulsion; AS37, a TLR-7 agonist adsorbed to Alum; CpG1018-Alum, a TLR-9 agonist formulated in Alum; and Alum. RBD-NP immunization with AS03, CpG1018-Alum, AS37 or Alum induced substantial nAb and CD4 T cell responses, and conferred protection against SARS-CoV-2 infection in the pharynges, nares and bronchoalveolar lavage. Live-virus nAb response was maintained up to 180 days post-vaccination with RBD/AS03, and correlated with protection. RBD-N

In [13]:
def paragraph_tokenize(sequence):
    if len(sequence.split('.\n')) < len(sequence.split('. \n')):
        paragraphs = sequence.split('. \n')
    else:
        paragraphs = sequence.split('.\n')
    print('Text split in:', len(paragraphs), 'paragraphs' )
    return paragraphs

In [14]:
paragraphs = paragraph_tokenize(sequence)

Text split in: 7 paragraphs


In [7]:
len(paragraphs[0])

1676

In [15]:
paragraphs = [paragraphs[0] + paragraphs[1] + paragraphs[2],paragraphs[3] + paragraphs[4] + paragraphs[5]]

['The development of a portfolio of COVID-19 vaccines to vaccinate the global population remains an urgent public health imperative1. Here we demonstrate the capacity of a subunit vaccine, comprising the SARS-CoV-2 spike receptor binding domain displayed on a protein nanoparticle (RBD-NP), to stimulate robust and durable neutralizing antibody (nAb) responses and protection against SARS-CoV-2 in non-human primates. We evaluated five adjuvants including Essai O/W 1849101, a squalene-in-water emulsion; AS03, an alpha-tocopherol-containing oil-in-water emulsion; AS37, a TLR-7 agonist adsorbed to Alum; CpG1018-Alum, a TLR-9 agonist formulated in Alum; and Alum. RBD-NP immunization with AS03, CpG1018-Alum, AS37 or Alum induced substantial nAb and CD4 T cell responses, and conferred protection against SARS-CoV-2 infection in the pharynges, nares and bronchoalveolar lavage. Live-virus nAb response was maintained up to 180 days post-vaccination with RBD/AS03, and correlated with protection. RBD

In [19]:
entities = class_entities.Entities(sequence, [], [], [])
offset = 0

for i,paragraph in enumerate(paragraphs):
    disease_service.sentence_to_process(paragraph)
    disease_service.set_offset(offset+i)
    disease_results = disease_service.predict()
    print(disease_service.offset)
    offset = len(paragraph)
    entities.append_entities(disease_results)

0
3199


In [20]:
entities_parsed = entities.parse_ner_spacy()

entities_html = displacy.render(entities_parsed, style="ent", manual=True,
                                options={"ents": ["DISEASE", "CHEMICAL"], "colors": colors})

In [16]:
tokenizer_disease = disease_service.tokenizer

def chunk_text(offset,tokenized_sequence):
    split_index = -1
    for j in range(offset+509,offset-1,-1):
        if tokenized_sequence[j] == '.':
            if tokenized_sequence[j+1][0].isupper():
                split_index = j
                break
    return split_index

print(len(tokenized_sequence))
processing = True
offset=0
tokenized_sequence = tokenizer_disease.tokenize(sequence)
index_list = []
while processing:
    if len(tokenized_sequence[offset:])<510:
        print(len(tokenized_sequence[offset:]))
        processing = True
        break
    split_index = chunk_text(offset, tokenized_sequence)
    print('Split index',split_index)
    print(len(tokenized_sequence[offset:]))
    offset = split_index + 1
    index_list.append(split_index)

chunks = []
for chunk in index_list:
    pattern = tokenized_sequence[chunk-1] + tokenized_sequence[chunk]
    result = re.search(pattern + ' ', sequence)
    print(result)
    if result is None:
        result = re.search(pattern + '\n', sequence)
    elif result is None:
        result = re.search(pattern, sequence)
    chunk_size = result.span()
    chunks.append(chunk_size[1])

start=0
entities = class_entities.Entities(sequence, [], [], [])
for i,chunk in enumerate(chunks):
    chemical_results = []
    chunk_number = i + 1
    if chunk_number<len(chunks):
        print(start)
        disease_service.sentence_to_process(sequence[start:chunk])    
        disease_service.set_offset(start)
        start = chunk
    else:
        print(start)
        print(sequence[start:])
        disease_service.sentence_to_process(sequence[start:])

    disease_results = disease_service.predict()
    

    gene_results = []
    

    entities.append_entities(disease_results)

In [128]:
chemical_service.sentence_to_process(sequence)
chemical_results = chemical_service.predict()

disease_service.sentence_to_process(sequence)
disease_results = disease_service.predict()

gene_results = []

entities = class_entities.Entities(sequence, disease_results, chemical_results, gene_results)

entities_parsed = entities.parse_ner_spacy()

entities_html = displacy.render(entities_parsed, style="ent", manual=True,
                                options={"ents": ["DISEASE", "CHEMICAL"], "colors": colors})

In [131]:
sequence[382:384]

'SA'